![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [55]:
import pandas as pd
import numpy as np

# Start coding here...
bank = pd.read_csv('bank_marketing.csv')
bank.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,13,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,19,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,23,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,27,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,3,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [56]:
# Split the dataframe
client = bank[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]

campaign = bank[['client_id', 'campaign', 'month', 'day', 'duration', 'pdays', 'previous', 'poutcome', 'y']]

economics = bank[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]

In [57]:
# Rename columns
client = client.rename(columns={'client_id':'id'})

campaign = campaign.rename(columns={'duration':'contact_duration', 'previous':'previous_campaign_contacts', 'y':'campaign_outcome', 'poutcome':'previous_outcome', 'campaign':'number_contacts'})

economics = economics.rename(columns={'euribor3m':'euribor_three_months', 'nr_employed':'number_employed'})

In [58]:
# Clean columns
client['education'] = client['education'].str.replace('.', '_')

client['education'] = client['education'].replace('unknow', np.NaN)

client['job'] = client['job'].str.replace('.', '')

In [59]:
def binary_outcome(outcome):
    return 1 if outcome == 'success' else 0

campaign['previous_outcome'] = campaign['previous_outcome'].apply(binary_outcome)

campaign['campaign_outcome'] = campaign['campaign_outcome'].apply(binary_outcome)

campaign['previous_outcome'] = campaign['previous_outcome'].replace('nonexistent', np.NaN)

campaign.head()

,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,13,261,999,0,0,0
1,1,1,may,19,149,999,0,0,0
2,2,1,may,23,226,999,0,0,0
3,3,1,may,27,151,999,0,0,0
4,4,1,may,3,307,999,0,0,0


In [60]:
# Create new columns
campaign['campaign_id'] = 1

campaign['year'] = '2022'

campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  int64 
 1   number_contacts             41188 non-null  int64 
 2   month                       41188 non-null  object
 3   day                         41188 non-null  int64 
 4   contact_duration            41188 non-null  int64 
 5   pdays                       41188 non-null  int64 
 6   previous_campaign_contacts  41188 non-null  int64 
 7   previous_outcome            41188 non-null  int64 
 8   campaign_outcome            41188 non-null  int64 
 9   campaign_id                 41188 non-null  int64 
 10  year                        41188 non-null  object
dtypes: int64(9), object(2)
memory usage: 3.5+ MB


In [61]:
campaign['last_contact_date'] = pd.to_datetime(campaign['year'] + '-' + bank['month'] + '-' + bank['day'].astype(str))

campaign.drop(['year', 'month', 'day'], axis=1, inplace=True)

campaign.head()

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,261,999,0,0,0,1,2022-05-13
1,1,1,149,999,0,0,0,1,2022-05-19
2,2,1,226,999,0,0,0,1,2022-05-23
3,3,1,151,999,0,0,0,1,2022-05-27
4,4,1,307,999,0,0,0,1,2022-05-03


In [62]:
# Save three dataframe
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)

In [63]:
# Create table
client_table = """CREATE TABLE client (
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOL,
    housing BOOL,
    loan BOOL
    );
    \copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """CREATE TABLE campaign(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOL,
    campaign_outcome BOOL,
    last_contact_date DATE
    );
    \copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """CREATE TABLE economics(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
    );
    \copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""